In [1]:
import pandas as pd

bus_data = pd.read_csv('DATA_daily_2017_2024.csv')
# '사용일자' 열을 datetime 형태로 변환
bus_data['사용일자'] = pd.to_datetime(bus_data['사용일자'], format='%Y%m%d')

# 2023년 1월 1일 이후 데이터만 필터링
bus_data = bus_data[bus_data['사용일자'] >= pd.Timestamp('2023-01-01')]
bus_data.reset_index(drop=True, inplace=True)
bus_data

,사용일자,승차총승객수,하차총승객수
0,2023-01-01,3581.0,3496.0
1,2023-01-02,7551.0,7400.0
2,2023-01-03,7804.0,7654.0
3,2023-01-04,8299.0,8100.0
4,2023-01-05,8026.0,7836.0
...,...,...,...
722,2024-12-27,8891.0,8712.0
723,2024-12-28,6985.0,6866.0
724,2024-12-29,5405.0,5358.0
725,2024-12-30,8464.0,8300.0


In [2]:
#승하차 인원 날짜별로 집계 : 일단 모든 정류장
daily_bus_data = bus_data.groupby('사용일자').agg({'승차총승객수': 'sum', '하차총승객수': 'sum'}).reset_index()
daily_bus_data.head(2)

,사용일자,승차총승객수,하차총승객수
0,2023-01-01,3581.0,3496.0
1,2023-01-02,7551.0,7400.0


In [3]:
daily_bus_data.rename(columns={
    '사용일자': 'Date',
    '승차총승객수': 'boarding_count',
    '하차총승객수': 'alighting_count',}, inplace=True)
daily_bus_data

,Date,boarding_count,alighting_count
0,2023-01-01,3581.0,3496.0
1,2023-01-02,7551.0,7400.0
2,2023-01-03,7804.0,7654.0
3,2023-01-04,8299.0,8100.0
4,2023-01-05,8026.0,7836.0
...,...,...,...
722,2024-12-27,8891.0,8712.0
723,2024-12-28,6985.0,6866.0
724,2024-12-29,5405.0,5358.0
725,2024-12-30,8464.0,8300.0


In [7]:
event_data1 = pd.read_csv('snu_crawling_2023.csv')
event_data2 = pd.read_csv('snu_crawling_2024.csv')

In [8]:
event_data2

,Date,Event
0,20240301,제1학기 시작
1,20240304,제1학기 개강
2,20240304,2024학년도 제2학기 교과목 수요조사
3,20240305,2024학년도 제2학기 교과목 수요조사
4,20240306,2024학년도 제2학기 교과목 수요조사
...,...,...
150,20241220,제2학기 보강기간
151,20241216,2025학년도 제1학기 복적 및 재입학 신청
152,20241216,2025학년도 제1학기 복학(귀) 신청
153,20241223,"동계휴가 시작, 동계계절수업 개강"


In [9]:
# 중복 날짜의 이벤트를 하나의 문자열로 합치기
event_data2 = event_data2.groupby('Date')['Event'].agg(', '.join).reset_index()
event_data2.tail(10)

,Date,Event
107,20241213,제2학기 종강
108,20241214,제2학기 보강기간
109,20241215,제2학기 보강기간
110,20241216,"제2학기 보강기간, 2025학년도 제1학기 복적 및 재입학 신청, 2025학년도 제..."
111,20241217,제2학기 보강기간
112,20241218,제2학기 보강기간
113,20241219,제2학기 보강기간
114,20241220,제2학기 보강기간
115,20241223,"동계휴가 시작, 동계계절수업 개강"
116,20241227,수업일수 1/4선


In [10]:
event_data = pd.concat([event_data1, event_data2], ignore_index=True)
event_data

,Date,Event
0,20230301,제1학기 시작
1,20230302,"제1학기 개강, 입학식, 제1학기 수강신청 변경"
2,20230309,"하계 계절수업 수요조사, 제2학기 개설교과목 신청"
3,20230323,하계 계절수업 개설교과목 신청
4,20230327,수업일수 1/4선
...,...,...
157,20241218,제2학기 보강기간
158,20241219,제2학기 보강기간
159,20241220,제2학기 보강기간
160,20241223,"동계휴가 시작, 동계계절수업 개강"


In [14]:
# event_data의 'Date' 열을 날짜형으로 변환
event_data['Date'] = pd.to_datetime(event_data['Date'], format='%Y%m%d', errors='coerce')

In [15]:
combined_data_all = pd.merge(daily_bus_data, event_data, on='Date', how='left')
combined_data_all

,Date,boarding_count,alighting_count,Event
0,2023-01-01,3581.0,3496.0,NaN
1,2023-01-02,7551.0,7400.0,NaN
2,2023-01-03,7804.0,7654.0,NaN
3,2023-01-04,8299.0,8100.0,NaN
4,2023-01-05,8026.0,7836.0,NaN
...,...,...,...,...
722,2024-12-27,8891.0,8712.0,수업일수 1/4선
723,2024-12-28,6985.0,6866.0,NaN
724,2024-12-29,5405.0,5358.0,NaN
725,2024-12-30,8464.0,8300.0,NaN


In [17]:
combined_data_all.to_csv('combined_data_2023_2024.csv', index=False)

In [18]:
df = pd.read_csv('combined_data_2023_2024.csv')
df

,Date,boarding_count,alighting_count,Event
0,2023-01-01,3581.0,3496.0,NaN
1,2023-01-02,7551.0,7400.0,NaN
2,2023-01-03,7804.0,7654.0,NaN
3,2023-01-04,8299.0,8100.0,NaN
4,2023-01-05,8026.0,7836.0,NaN
...,...,...,...,...
722,2024-12-27,8891.0,8712.0,수업일수 1/4선
723,2024-12-28,6985.0,6866.0,NaN
724,2024-12-29,5405.0,5358.0,NaN
725,2024-12-30,8464.0,8300.0,NaN
